In [1]:
# -*- coding: utf-8 -*-

'''
@Author   :   Corley Tang
@contact  :   cutercorleytd@gmail.com
@Github   :   https://github.com/corleytd
@Time     :   2023-01-15 18:29
@Project  :   Hands-on Deep Learning with PyTorch-single_layer_neural_network
单层神经网络
'''

# 导入所需的库
import warnings

import torch
from torch import nn
from torch.nn import functional as F

warnings.filterwarnings('ignore', category=UserWarning)

## 1.PyTorch中的单层回归网络：线性回归
### 理论基础
线性回归的任务是构造一个预测函数来映射输入的特征矩阵X和标签值y的线性关系。这个预测函数的图像是一条直线，所以线性回归的求解就是对直线的拟合过程。这个预测函数的本质就是我们需要构建的模型，而**构造预测函数的核心就是找出模型的权重向量**，也就是求解线性方程组的参数。
一个单层线性回归的神威网络示意图如下：
![single_lr_nn](../assets/single_lr_nn.png)
### tensor实现单层神经网络的正向传播

In [2]:
# torch.tensor：会根据输入的数据类型来确定tensor的数据类型
# torch.Tensor：无论输入的数据类型是什么，张量的数据类型都是float32，不建议使用
X = torch.tensor([[1., 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]])
z = torch.tensor([-0.2, -0.05, -0.05, 0.1])
w = torch.tensor([-0.2, 0.15, 0.15])
X, z, w

(tensor([[1., 0., 0.],
         [1., 0., 1.],
         [1., 1., 0.],
         [1., 1., 1.]]),
 tensor([-0.2000, -0.0500, -0.0500,  0.1000]),
 tensor([-0.2000,  0.1500,  0.1500]))

In [3]:
# 线性回归函数
def linear_regression(X, w):
    z_hat = torch.mv(X, w)  # 矩阵乘向量
    return z_hat

In [4]:
# 调用
z_hat = linear_regression(X, w)
z_hat

tensor([-0.2000, -0.0500, -0.0500,  0.1000])

### PyTorch新手避坑指南
1.PyTorch的静态性
- 很多函数要求两个张量的数据类型必须一致
- 许多函数都不接受浮点型的分类标签，但也有许多函数要求真实标签的类型必须与预测值的类型一致，因此标签的类型定义总是一个容易踩坑的地方
- PyTorch中许多函数不接受一维张量，但同时也有许多函数不接受二维标签，因此在生成标签时可以默认生成二维标签

In [5]:
# 2.精度问题
z == z_hat  # 看起来预测值和真实值相等，实际上不等

tensor([ True, False, False, False])

In [6]:
(z_hat - z).square().sum()  # SSE的值也不为0

tensor(8.3267e-17)

In [7]:
# 设置显示精度，来查看比较预测值和真实值
torch.set_printoptions(precision=30)  # 看小数点后30位

z_hat, z  # 两者有精度的差异：float32本身带来的精确度损失，mv函数内部计算带来的微笑精度问题

(tensor([-0.200000002980232238769531250000, -0.049999997019767761230468750000,
         -0.049999997019767761230468750000,  0.100000008940696716308593750000]),
 tensor([-0.200000002980232238769531250000, -0.050000000745058059692382812500,
         -0.050000000745058059692382812500,  0.100000001490116119384765625000]))

In [8]:
# 放大精度问题
preds = torch.ones(300, 68, 64, 64)
preds.shape, preds.numel(), preds.sum()

(torch.Size([300, 68, 64, 64]), 83558400, tensor(83558400.))

In [9]:
preds *= 0.1
preds.sum() * 10  # 两者计算结果不同，带来了较明显的精度问题

tensor(83558328.)

In [10]:
# 要缓解精度问题，可以使用float64位
preds = torch.ones(300, 68, 64, 64, dtype=torch.float64)
preds.shape, preds.numel(), preds.sum()

(torch.Size([300, 68, 64, 64]),
 83558400,
 tensor(83558400., dtype=torch.float64))

In [11]:
preds *= 0.1
preds.sum() * 10  # 两者计算结果不同，但是精度问题得到了很大程度的缓解

tensor(83558400.000000059604644775390625000000, dtype=torch.float64)

In [12]:
# 无视微小区别，比较两个张量是否一致
torch.allclose(z_hat, z)

True

### torch.nn.Linear实现单层回归神经网络的正向传播
torch.nn是包含了构筑神经网络结构基本元素的包，这个包中可以找到任意的神经网络层，这些神经网络层都是nn.Module这个大类的子类，torch.nn.Linear就是神经网络中的”线性层“。

In [13]:
# 重新定义数据
X = torch.tensor([[0., 0], [0, 1], [1, 0], [1, 1]])  # 输入Linear层的数据不需要添加x0
z = torch.tensor([-0.2, -0.05, -0.05, 0.1])
torch.set_printoptions(precision=4)
X, z

(tensor([[0., 0.],
         [0., 1.],
         [1., 0.],
         [1., 1.]]),
 tensor([-0.2000, -0.0500, -0.0500,  0.1000]))

In [14]:
# 实例化Linear层对象，默认带bias
linear = nn.Linear(2, 1)  # 参数：上一层的神经元个数（特征个数），这一层的神经元个数
linear, linear.weight, linear.bias  # 查看Linear对象及随机生成的权重和偏差

(Linear(in_features=2, out_features=1, bias=True),
 Parameter containing:
 tensor([[ 0.6152, -0.1616]], requires_grad=True),
 Parameter containing:
 tensor([0.1680], requires_grad=True))

In [15]:
# 前向传播，得到预测值
z_hat = linear(X)
z_hat

tensor([[0.1680],
        [0.0063],
        [0.7832],
        [0.6215]], grad_fn=<AddmmBackward0>)

In [16]:
# 实例化Linear层对象，不带bias
linear = nn.Linear(2, 1, bias=False)
linear, linear.weight, linear.bias

(Linear(in_features=2, out_features=1, bias=False),
 Parameter containing:
 tensor([[-0.1931,  0.4021]], requires_grad=True),
 None)

In [17]:
z_hat = linear(X)  # 神经网络不需要定义输入层
z_hat

tensor([[ 0.0000],
        [ 0.4021],
        [-0.1931],
        [ 0.2090]], grad_fn=<MmBackward0>)

In [18]:
# 设置随机数种子，控制每次生成的权重为固定值
torch.random.manual_seed(2023)  # 人为设置随机数种子
linear = nn.Linear(2, 1)
linear, linear.weight, linear.bias

(Linear(in_features=2, out_features=1, bias=True),
 Parameter containing:
 tensor([[-0.1004,  0.3112]], requires_grad=True),
 Parameter containing:
 tensor([0.6338], requires_grad=True))

In [19]:
torch.random.manual_seed(2023)
linear = nn.Linear(2, 1)
linear, linear.weight, linear.bias  # 可复现

(Linear(in_features=2, out_features=1, bias=True),
 Parameter containing:
 tensor([[-0.1004,  0.3112]], requires_grad=True),
 Parameter containing:
 tensor([0.6338], requires_grad=True))

In [20]:
linear = nn.Linear(2, 1)
linear, linear.weight, linear.bias  # 与前面不一样，因为种子固定的是随机数，而不是随机数本身

(Linear(in_features=2, out_features=1, bias=True),
 Parameter containing:
 tensor([[-0.0288,  0.0585]], requires_grad=True),
 Parameter containing:
 tensor([0.6938], requires_grad=True))

## 2.二分类神经网络：逻辑回归
### 理论基础
线性关系描述的是变量之间的线性关系，但实际上变量之间更多的是曲线关系，为了更好地拟合曲线关系，在线性方程中引入**联系函数**，称为**广义线性回归**，包括对数线性回归、S形函数回归等，最经典的还是对数几率回归， 即逻辑回归，其在线性方程中引入Sigmoid函数，函数公式为$\sigma=\operatorname{Sigmoid}(z)=\frac{1}{1+e^{-z}}$，函数图像如下：
![sigmoid_curve](../assets/sigmoid_curve.png)

可以看到，Sigmoid函数有一些特殊的性质：
- 当自变量z趋近正无穷时，因变量σ趋近于1，而当z趋近负无穷时，σ趋近于0，这使得Sigmoid函数能够将任何实数映射到(0,1)区间
- Sigmoid的导数在0点时最大（这一点的斜率最大），所以它可以快速将数据从0的附近排开，让数据点到远离自变量取0的地方去，让sigmoid函数拥有将连续性变量z转化为离散型变量σ的力量，这也就是化回归算法为分类算法的力量，具体给σ设定阈值即可，例如0.5
- σ取对数几率后的结果就是线性回归的z，因此这个算法被称为对数几率回归，也就是逻辑回归

### tensor实现二分类神经网络的正向传播
Sigmoid二分类神经网络示意图如下：

![sigmoid_nn](../assets/sigmoid_nn.png)

In [21]:
# 具体实现
X = torch.tensor([[1., 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]])
and_gate = torch.tensor([0., 0, 0, 1])
w = torch.tensor([-0.2, 0.15, 0.15])
X, and_gate, w  # 与门数据

(tensor([[1., 0., 0.],
         [1., 0., 1.],
         [1., 1., 0.],
         [1., 1., 1.]]),
 tensor([0., 0., 0., 1.]),
 tensor([-0.2000,  0.1500,  0.1500]))

In [22]:
# 自定义逻辑回归
def logistic_regression(X, w, prob=0.5):
    z_hat = X @ w
    # sigma = torch.sigmoid(z_hat)  # 自带的Sigmoid函数
    sigma = 1 / (1 + torch.exp(-z_hat))
    preds = (sigma >= prob).float()  # 等价于torch.tensor([float(t) for t in (sigma >= prob)])
    return sigma, preds

In [23]:
sigma, preds = logistic_regression(X, w)
sigma, preds

(tensor([0.4502, 0.4875, 0.4875, 0.5250]), tensor([0., 0., 0., 1.]))

In [24]:
# 比较预测值和真实值
preds == and_gate

tensor([True, True, True, True])

### 其他可以将连续型数据分割为离散型数据的函数
除了Sigmoid函数，还有许多其他的函数可以被用来将连续型数据分割为离散型数据。
**1.符号函数sign**
公式为：
$$y=\left\{\begin{aligned}
1 & \text { if } z>0 \\
0 & \text { if } z=0 \\
-1 & \text { if } z<0
\end{aligned}\right.$$
函数图像如下：

![sign_curve](../assets/sign_curve.png)

由于函数的取值是间断的，符号函数也被称为**阶跃函数**，表示在0的两端，函数的结果y是从-1直接阶跃到了1。因为输出结果直接是0、1、-1这样的类别，所以可以直接用于分类。在二分类中，符号函数也可以忽略中间的0，直接分为0和1两类，如下：
$$y=\left\{\begin{array}{ll}
1 & \text { if } z>0 \\
0 & \text { if } z \leq 0
\end{array}\right.$$
即
$$y=\left\{\begin{array}{ll}
1 & \text { if } w_{1} x_{1}+w_{2} x_{2}+b>0 \\
0 & \text { if } w_{1} x_{1}+w_{2} x_{2}+b \leq 0
\end{array}\right.$$

In [25]:
# 用阶跃函数实现二分类
def logistic_regression_with_sign(X, w):
    z_hat = X @ w
    preds = (z_hat >= 0).float()
    return z_hat, preds

In [26]:
# 调用
z_hat, preds = logistic_regression_with_sign(X, w)
z_hat, preds

(tensor([-0.2000, -0.0500, -0.0500,  0.1000]), tensor([0., 0., 0., 1.]))

**2.ReLU**

ReLU(Rectified Linear Unit)函数又名整流线型单元函数，提供了一个很简单的非线性变换：当输入的自变量大于0时，直接输出该值，当输入的自变量小于等于0时，输出0。公式如下：
$$\operatorname{Re} L U: \sigma=\left\{\begin{array}{ll}
z & (z>0) \\
0 & (z \leq 0)
\end{array}\right.$$
可以看到，ReLU函数的本质就是max(0,z)，从输入的数值中选择较大的那个值进行输出，以达到保留正数元素、将负元素清零的作用。函数图像如下：
![relu_curve](../assets/relu_curve.png)

ReLU函数导数的图像如下：

![relu_derivative](../assets/relu_derivative.png)

可以看到，ReLU函数的导数就是阶跃函数，当输入 为正数时，ReLU函数的导数为1，当z为负数时，ReLU函数的导数为0，当输入为0时，ReLU函数不可导。

**3.Tanh**

Tanh（hyperbolic tangent）是双曲正切函数，性质与Sigmoid相似，能够将数值压缩到(-1, 1)区间内。公式如下：
$$\tanh : \sigma=\frac{e^{2 z}-1}{e^{2 z}+1}$$
函数图像如下：

![tanh_curve](../assets/tanh_curve.png)

显然，Tanh的图像和Sigmoid函数很像，区别在于，Sigmoid函数的范围是在(0, 1)之间，Tanh却是在坐标系的原点(0, 0)点上中心对称。Tanh函数的导数表达式为$\tanh ^{\prime}(z)=1-\tanh ^{2}(z)$，图像如下：

![tanh_derivative](../assets/tanh_derivative.png)

可以看到，当输入的z约接近于0，Tanh函数导数也越接近最大值1，当输入越偏离0时，Tanh函数的导数越接近于0，也起到了类似Sigmoid函数的将将数据从0的附近排开的作用，从而将连续型变量转变为类别变量、实现二分类任务。这些函数是最常见的二分类转化函数，他们在神经网络的结构中有着不可替代的作用，也被称为**激活函数**。
### torch.functional实现单层二分类神经网络的正向传播
逻辑回归与线性回归的唯一区别，就是在线性回归的结果之后套上了Sigmoid函数，因此只要让nn.Linear的输出结果再经过sigmoid函数，就可以实现逻辑回归的正向传播。在PyTorch中，几乎总是从nn.functional中调用相关函数。

In [27]:
X = torch.tensor([[0., 0], [0, 1], [1, 0], [1, 1]])
X

tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]])

In [28]:
# 实例化和调用线性层
prob = 0.5  # 概率阈值
linear = nn.Linear(2, 1)
z_hat = linear(X)
sigma = F.sigmoid(z_hat)  # 等价于torch.sigmoid(z_hat)
y = (sigma >= prob).int().squeeze().tolist()
z_hat, y

(tensor([[-0.4478],
         [-0.8462],
         [-0.5771],
         [-0.9754]], grad_fn=<AddmmBackward0>),
 [0, 0, 0, 0])

In [29]:
# 使用符号函数
torch.sign(z_hat)  # F.sign(z_hat)：AttributeError，符号函数只能使用torch调用，相比于神经网络元素，更像一个数学公式

tensor([[-1.],
        [-1.],
        [-1.],
        [-1.]], grad_fn=<SignBackward0>)

In [30]:
# 使用ReLU函数
torch.relu(z_hat), F.relu(z_hat)

(tensor([[0.],
         [0.],
         [0.],
         [0.]], grad_fn=<ReluBackward0>),
 tensor([[0.],
         [0.],
         [0.],
         [0.]], grad_fn=<ReluBackward0>))

In [31]:
# 使用Tanh函数
torch.tanh(z_hat), F.tanh(z_hat)  # Sigmoid、ReLU和Tanh等函数可以使用functional来调用，因为它们是对神经网络元素更有用的功能

(tensor([[-0.4201],
         [-0.6891],
         [-0.5205],
         [-0.7511]], grad_fn=<TanhBackward0>),
 tensor([[-0.4201],
         [-0.6891],
         [-0.5205],
         [-0.7511]], grad_fn=<TanhBackward0>))

## 3.多分类神经网络：Softmax回归
### 认识Softmax函数
实际生活中，出了二分类问题，还有很多多分类问题，例如手写数字识别就是一个十分类问题。在机器学习中，我们会使用二分类算法的Many-vs-Many（多对多）和One-vs-Rest（一对多）模式来进行多分类，但在深度学习中并不适用，此时有更好的方法——Softmax回归。Softmax函数是深度学习基础中的基础，它是神经网络进行多分类时，默认放在输出层中处理数据的函数，公式如下：
$$\sigma_{k}=\operatorname{Softmax}\left(z_{k}\right)=\frac{e^{z_{k}}}{\sum^{K} e^{z}}$$
可以看到，Softmax函数的分子是多分类状况下某一个标签类别的回归结果的指数函数，分母是多分类状况下所有标签类别的回归结果的指数函数之和，因此Softmax函数的结果代表了样本的结果为类别k的概率。
例如对于一个三分类任务，神经网络结构如下：
![softmax_nn](../assets/softmax_nn.png)

在多分类中，神经元的个数与标签类别的个数是一致的，如果是十分类，在输出层上就会存在十个神经元，分别输出十个不同的概率。上图中，样本的预测标签就是输出的3个概率中最大的概率对应的标签类别。
### PyTorch中的Softmax函数
Softmax函数可以将多分类的结果转变为概率，但它需要的计算量非常巨大：由于Softmax的分子和分母中都带有e为底的指数函数，所以在计算中如果线性回归输出的z非常大，则$e^z$会非常大，甚至为inf，导致**溢出**，使得Softmax操作失效。

In [32]:
# 大数据量时的Softmax失效
z1 = torch.tensor([10., 9, 8])
z2 = torch.tensor([105., 100, 95])
z3 = torch.tensor([1010., 1000, 990])
z1.exp() / z1.exp().sum(), z2.exp() / z2.exp().sum(), z3.exp() / z3.exp().sum()  # Softmax的自定义实现

(tensor([0.6652, 0.2447, 0.0900]),
 tensor([nan, nan, nan]),
 tensor([nan, nan, nan]))

In [33]:
# 使用torch.softmax时，需要传入维度的索引，指定在哪一个维度进行Softmax
F.softmax(z1), torch.softmax(z2, 0), F.softmax(z3, -1)  # Softmax函数可以通过functional和torch两种方式调用，只不过参数要求不完全一致

(tensor([0.6652, 0.2447, 0.0900]),
 tensor([9.9326e-01, 6.6925e-03, 4.5094e-05]),
 tensor([9.9995e-01, 4.5398e-05, 2.0611e-09]))

显然，PyTorch自带的Softmax使用了一些巧妙的手段来解决溢出问题。同时，输入的z越大，经过Softmax之后的概率值也越大，无论是否使用Softmax，都可以判断出样本被预测为哪一类，这是因为$e^z$是单调递增函数。所以在如果不需要具体概率、只需要分类出的类别结果时，此时可以不在最后一层加Softmax，一般的处理方式是在训练时加Softmax、测时时不加Softmax

In [34]:
# 不同维度的Softmax操作：指定在哪一个维度进行操作，结果就是将哪一个维度上的大小作为类别数、计算Softmax概率
t1 = torch.randint(1, 6, size=(3, 4, 5), dtype=torch.float)
torch.softmax(t1, dim=0), F.softmax(t1, 1), t1.softmax(-1)

(tensor([[[0.8668, 0.0453, 0.4683, 0.0177, 0.4223],
          [0.5761, 0.7054, 0.9094, 0.2119, 0.0171],
          [0.1554, 0.8438, 0.9362, 0.4223, 0.2595],
          [0.1554, 0.9362, 0.0453, 0.0453, 0.4879]],
 
         [[0.1173, 0.0453, 0.0634, 0.0177, 0.1554],
          [0.2119, 0.2595, 0.0453, 0.2119, 0.9362],
          [0.4223, 0.0420, 0.0171, 0.4223, 0.0351],
          [0.4223, 0.0466, 0.0453, 0.9094, 0.0243]],
 
         [[0.0159, 0.9094, 0.4683, 0.9647, 0.4223],
          [0.2119, 0.0351, 0.0453, 0.5761, 0.0466],
          [0.4223, 0.1142, 0.0466, 0.1554, 0.7054],
          [0.4223, 0.0171, 0.9094, 0.0453, 0.4879]]]),
 tensor([[[0.6572, 0.0279, 0.0628, 0.0130, 0.0347],
          [0.2418, 0.2060, 0.4643, 0.2619, 0.0128],
          [0.0120, 0.2060, 0.4643, 0.7120, 0.2562],
          [0.0889, 0.5601, 0.0085, 0.0130, 0.6964]],
 
         [[0.1966, 0.1966, 0.1749, 0.0104, 0.0164],
          [0.1966, 0.5344, 0.4754, 0.2097, 0.8945],
          [0.0723, 0.0723, 0.1749, 0.5701, 0.0445],


### 使用Linear层和functional实现多分类神经网络
使用线性层和Softmax实现多分类神经网络需要确定以下几点：
- 线性层的初始化参数：in_features=2, out_features=3
- Softmax的使用维度：最后一个维度
- 输出值的维度：(4, 3)

In [35]:
# 具体实现
X = torch.tensor([[0., 0], [0, 1], [1, 0], [1, 1]])
linear = nn.Linear(2, 3)
z_hat = linear(X)
sigma = F.softmax(z_hat, -1)
z_hat, sigma

(tensor([[ 0.4729,  0.4893, -0.4421],
         [-0.2022,  0.4964, -0.9949],
         [ 0.0325,  1.1336, -0.9684],
         [-0.6426,  1.1407, -1.5212]], grad_fn=<AddmmBackward0>),
 tensor([[0.4137, 0.4206, 0.1657],
         [0.2887, 0.5806, 0.1307],
         [0.2286, 0.6874, 0.0840],
         [0.1358, 0.8078, 0.0564]], grad_fn=<SoftmaxBackward0>))